In [ ]:
import pandas as pd
import numpy as np

# Load the data
data = pd.read_csv('../data/driving_data.csv')

# Feature Engineering
data['Speed_diff'] = data['Speed(km/h)'].diff().fillna(0)
data['Acceleration_diff'] = data['Acceleration(m/s^2)'].diff().fillna(0)
data['Heading_diff'] = data['Heading_Change(degrees)'].diff().fillna(0)

# Aggregation to create trip-level summaries
trip_summary = data.groupby('TripId').agg({
    'Speed(km/h)': ['mean', 'max', 'min'],
    'Acceleration(m/s^2)': ['mean', 'max', 'min'],
    'Heading_Change(degrees)': ['mean', 'max', 'min'],
    'Speed_diff': 'mean',
    'Acceleration_diff': 'mean',
    'Heading_diff': 'mean'
})

# Flatten multi-level columns
trip_summary.columns = ['_'.join(col).strip() for col in trip_summary.columns.values]
trip_summary.reset_index(inplace=True)

# Save the processed data
trip_summary.to_csv('../data/trip_summary.csv', index=False)
print("Data preprocessing complete. Processed data saved.")
